In [2]:
from IPython.display import Image
from IPython.core.display import HTML 


Entrenamiento provedido en Español <br>
Training offered in English

<img src="https://www.ciesin.columbia.edu/images/logo_sm.png" >

# ANALYSIS DE PUNTOS CRITICOS DE VULNERABILIDAD - AmeriGEO 2023 San Jose
# HOTSPOT VULNERABILITY ANALYSIS - AmeriGEO 2023 San Jose

### JUAN F. MARTINEZ, Center for International Earth Science Information Network (CIESIN)

#### The Hotspot vulnerability training will provide a comprehensive training in the framework, data, and methods used to develop a spatial vulnerability assessment.
##### The training will utilize open-acccess software and data to demonstrate how to develop a final hotspot analysis product using different sources and formats of data. The training will take three stages:
##### 1. Theoretical Framework of the Index Approach to Hotspospot Vulnerability Mapping 
##### 2. Processing of Different Data Products
##### 3. Indexing and Aggregating Process

## 1. Theoretical Framework of the Index Approach
The United Nation's 2030 Sustainable Development Agenda has provided us a blueprint for peace and prosperity for people and the planet, now and into the future. <br>
The 17 Sustainable Development Goals (SDG's) are the main pillars of the Agenda that call for global partnership from all member countries. <br>
<img src="https://www.undp.org/sites/g/files/zskgke326/files/migration/mm/sdg-sticker-eng.jpg" width="50%"><br>
##### The first Sustainable Development Goal (SDG) in the United Nations (UN) 2030 Agenda seeks to end poverty of all forms everywhere by reaching the poorest of the poor, and to ensure progress for all population groups. <br>
##### Poverty has been traditionally measured by individual economic dimensions of income and wealth. For example, the <a href="https://www.worldbank.org/en/understanding-poverty#:~:text=Regions%20are%20categorized%20using%20PIP%20definition.&text=Note%20on%20global%20poverty%20lines,%242.15%20per%20person%20per%20day." target="_blank">World Bank defined the global extreme-poverty line as $2.15 USD (2022)</a> income or below per day, which is based on the national poverty lines of the world’s 15 poorest countries. However, these indicators do not entirely consider other non-financial forms of poverty that may be of equal importance. The concept of multidimensional poverty encompasses a more holistic view because it attempts to account for the lived experiences of people and the multiple deprivations they face in their daily lives beyond their incomes.

## Examples of Multidimensional Poverty Indices


  
